In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pygame
import time
import gymnasium as gym
from gymnasium.spaces import Dict, Discrete, Box
from collections import OrderedDict


In [ ]:
class ObservableDeformedGridworld(gym.Env):

    def __init__(self, grid_size=(1.0, 1.0), step_size=0.02, goal=(0.9, 0.9), 
                 obstacles=None, stretch=(1.0, 1.0), shear=(0.0, 0.0), observation_radius=0.05, render_mode=None):
        """
        Initialize the observable deformed continuous gridworld.
        :param grid_size: Size of the grid (width, height).
        :param step_size: Step size for the agent's movement.
        :param goal: Coordinates of the goal position.
        :param obstacles: List of obstacles as rectangles [(x_min, y_min), (x_max, y_max)].
        :param stretch: Tuple (s_x, s_y) for stretching the grid in x and y directions.
        :param shear: Tuple (sh_x, sh_y) for shearing the grid.
        :param observation_radius: Radius within which the agent can observe its surroundings.
        """
        self.grid_size = np.array(grid_size)
        self.step_size = step_size
        self.goal = np.array(goal)
        self.state = np.array([0.0, 0.0])  # Start at the origin
        self.obstacles = obstacles if obstacles else []
        self.observation_radius = observation_radius

        # Transformation matrix
        self.transformation_matrix = np.array([
            [stretch[0], shear[0]],
            [shear[1], stretch[1]]
        ])
        self.inverse_transformation_matrix = np.linalg.inv(self.transformation_matrix)

        # Rendering mode
        self.render_mode = render_mode

        # gymnasium compatibility
        self.action_space = gym.spaces.Discrete(4)
        self.observation_space =  Dict({
            "pos": gym.spaces.Box(low=.0, high=1.0, shape=(2,),dtype=float),
            "theta": gym.spaces.Box(low=.0, high=1.0, shape=(4,),dtype=float), # deformation is a 2x2 tensor
        })

        self.stretch_range = np.array([0.4, 1])
        self.shear_range = np.array([-0.2, 0.2])

        self.timestep = 0
        
    def reset(self,seed=55):
        """
        Reset the environment to the initial state.
        :return: Initial state and observation.
        """
        np.random.seed(seed)
        self.state = np.random.rand(2) * self.grid_size  # Random start position
        self.set_deformation(self.sample(2,self.stretch_range), self.sample(2,self.shear_range))  # Reset deformation to random

        state = OrderedDict({
            "pos": self.transform(self.state),
            "theta": self.transformation_matrix.flatten()
        }) 
        
        self.timestep = 0

        # print(f"Initial agent position: {self.state}",
        #       f"Initial goal position: {self.goal}",
        #       f"Initial deformation: {self.transformation_matrix}",
        #       f"Initial observation: {self.observe_obstacle()}",
        #       sep="\n")
        
        return state, {}
    
    def set_deformation(self, stretch, shear):
        """
        Set the deformation transformation matrix based on stretch and shear parameters.
        
        This function creates a transformation matrix to apply grid deformations, including 
        stretching and shearing, to the grid coordinates. It also computes the inverse of 
        this transformation for reversing the deformation.

        :param stretch: A tuple (s_x, s_y) for stretching the grid in the x and y directions.
        :param shear: A tuple (sh_x, sh_y) for shearing the grid in the x and y directions.
        """
        # Create the transformation matrix based on stretch and shear
        self.transformation_matrix = np.array([
            [stretch[0], shear[0]],  # First row: stretch in x and shear in x direction
            [shear[1], stretch[1]]   # Second row: shear in y and stretch in y direction
        ])

        # Calculate the inverse transformation matrix for reversing the deformation
        self.inverse_transformation_matrix = np.linalg.inv(self.transformation_matrix)

        # Optionally, print the transformation matrices for debugging
        # print(f"Transformation Matrix:\n{self.transformation_matrix}")
        # print(f"Inverse Transformation Matrix:\n{self.inverse_transformation_matrix}")

    def set_pos(self, pos):
        """
        Set the agent's state to a new position.
        
        This function directly updates the agent's position (state) to the provided coordinates.

        :param pos: A tuple or array representing the new position of the agent in the grid.
        """
        # Update the state (agent's position)
        self.state = np.array(pos)

        # Optionally, print the new state for debugging
        # print(f"New agent position: {self.state}")

    def transform(self, position):
        """
        Apply the grid deformation to a given position.
        :param position: (x, y) in original space.
        :return: Transformed position in the deformed grid.
        """
        return np.dot(self.transformation_matrix, position)

    def inverse_transform(self, position):
        """
        Map a position from the deformed grid back to the original space.
        :param position: (x, y) in the deformed grid.
        :return: Original position.
        """
        return np.dot(self.inverse_transformation_matrix, position)

    def is_in_obstacle(self, position):
        """
        Check if a given position is inside any obstacle.
        :param position: The (x, y) coordinates to check in the original space.
        :return: True if the position is inside an obstacle, False otherwise.
        """
        for obs in self.obstacles:
            (x_min, y_min), (x_max, y_max) = obs
            if x_min <= position[0] <= x_max and y_min <= position[1] <= y_max:
                return True
        return False

    def observe_single_obstacle(self):
        """
        Check if any part of an obstacle is within the observation radius of the agent.
        :return: True if any part of an obstacle is within the observation radius, False otherwise.
        """
        for obs in self.obstacles:
            (x_min, y_min), (x_max, y_max) = obs
            
            # Clamp the agent's position to the obstacle's boundaries to find the closest point
            closest_x = np.clip(self.state[0], x_min, x_max)
            closest_y = np.clip(self.state[1], y_min, y_max)
            
            # Compute the distance from the agent to this closest point
            closest_point = np.array([closest_x, closest_y])
            distance_to_obstacle = np.linalg.norm(self.state - closest_point)
            
            # Check if this distance is within the observation radius
            if distance_to_obstacle <= self.observation_radius:
                return 1
        
        return 0

    def observe_obstacle(self):
        """
        Check if there are obstacles in the four cardinal directions (N, E, S, W).
        Each direction checks for obstacles in a quarter-circle arc within the observation radius.
        :return: A numpy array of shape (4,), where each entry indicates the presence of obstacles in 
                the respective direction (North, East, South, West).
        """
        directions = ["N", "E", "S", "W"]
        obstacle_presence = np.zeros(4)  # Default: no obstacles in any direction
        
        for obs in self.obstacles:
            (x_min, y_min), (x_max, y_max) = obs
            # Generate obstacle sample points for finer checks
            x_samples = np.linspace(x_min, x_max, 55)
            y_samples = np.linspace(y_min, y_max, 55)
            obstacle_points = np.array(np.meshgrid(x_samples, y_samples)).T.reshape(-1, 2)
            
            for point in obstacle_points:
                vector_to_point = point - self.state
                distance = np.linalg.norm(vector_to_point)
                if distance > self.observation_radius:
                    continue  # Ignore points outside the observation radius

                # Compute angle relative to positive Y-axis
                angle = np.arctan2(vector_to_point[1], vector_to_point[0]) * 180 / np.pi
                angle = (angle + 360) % 360  # Normalize angle to [0, 360)

                # Check which quarter-circle (arc) the point falls into
                if 315 <= angle or angle < 45:  # North
                    obstacle_presence[1] = 1
                elif 45 <= angle < 135:  # East
                    obstacle_presence[0] = 1
                elif 135 <= angle < 225:  # South
                    obstacle_presence[3] = 1
                elif 225 <= angle < 315:  # West
                    obstacle_presence[2] = 1

        return obstacle_presence

    def step(self, action):
        """
        Take a step in the environment, interpreting the action in the deformed space.
        :param action: One of ['N', 'S', 'E', 'W'].
        :return: Tuple (next_state, observation, reward, done, info).
        """
        # Map actions to movements in the deformed space
        moves_deformed = [np.array([0, self.step_size]),   # Move up in deformed space
            np.array([0, -self.step_size]),  # Move down in deformed space
            np.array([self.step_size, 0]),   # Move right in deformed space
            np.array([-self.step_size, 0])   # Move left in deformed space
        ]

        # Get the movement vector in the deformed space
        move_deformed = moves_deformed[action]

        # Map the movement to the original space using the inverse transformation
        move_original = np.dot(self.inverse_transformation_matrix, move_deformed)

        # Update state in the original grid space
        next_state = self.state + move_original
        # next_state = np.clip(next_state, [0, 0], self.grid_size)

        num_samples = 50  # Number of points to sample along the path
        path = np.linspace(self.state, next_state, num_samples)

        # Check for collisions along the path
        collision = any(self.is_in_obstacle(point) for point in path)

        # Check if the new state is in an obstacle
        if collision:
            reward = -1.0  # Penalty for hitting an obstacle
            info = {"collision": True}
            terminated = False
        # Check if the is inside the grid boundaries
        elif np.any(next_state != np.clip(next_state, [0, 0], self.grid_size)):
            reward = -1.0
            info = {"out": True}
            next_state = np.clip(next_state, [0, 0], self.grid_size)
            terminated = False
        else:   
            transformed_goal = self.transform(self.goal)
            transformed_state = self.transform(self.state)
            terminated = np.linalg.norm(transformed_state - transformed_goal) < self.step_size
            reward = 1.0 if terminated else -0.01
            info = {"collision": False, "out": False}
    
        self.state = next_state
        self.timestep += 1
        truncated = self.timestep >= 200 

        if self.render_mode == "human":
            self.render()

        state = OrderedDict({
                    "pos": self.transform(self.state),
                    "theta": self.transformation_matrix.flatten()
                }) 

        # Return the transformed state, reward, and terminated truncated flag
        return state, reward, terminated, truncated, info

    def render(self):
        """
        Render the deformed gridworld environment using Pygame, ensuring the entire deformed grid fits within the screen.
        """
        import pygame  # Ensure Pygame is imported

        # Define colors
        WHITE = (255, 255, 255)
        BLUE = (0, 0, 255)
        GREEN = (0, 255, 0)
        RED = (255, 0, 0)
        YELLOW = (255, 255, 0)
        BLACK = (0, 0, 0)

        # Initialize the screen
        if not hasattr(self, "screen"):
            self.screen_width = 800
            self.screen_height = 600
            pygame.init()
            self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
            pygame.display.set_caption("Deformed Gridworld Environment")

        # Fill background with white
        self.screen.fill(WHITE)

        # Compute the bounding box of the deformed grid
        corners = [
            self.transform(np.array([0, 0])),
            self.transform(np.array([self.grid_size[0], 0])),
            self.transform(self.grid_size),
            self.transform(np.array([0, self.grid_size[1]])),
        ]
        x_coords, y_coords = zip(*corners)
        min_x, max_x = min(x_coords), max(x_coords)
        min_y, max_y = min(y_coords), max(y_coords)

        # Define scaling factors to fit the deformed grid within the screen
        scale_x = self.screen_width / (max_x - min_x)
        scale_y = self.screen_height / (max_y - min_y)
        scale = min(scale_x, scale_y)  # Uniform scaling to maintain aspect ratio

        # Transform helper for rendering
        def to_screen_coords(pos):
            """
            Map transformed coordinates to screen coordinates, scaled and shifted to fit the screen.
            """
            x, y = pos
            x_screen = int((x - min_x) * scale)
            y_screen = int((max_y - y) * scale)  # Flip y-axis for screen rendering
            return x_screen, y_screen

        # Draw the deformed grid boundaries
        pygame.draw.polygon(self.screen, BLACK, [to_screen_coords(corner) for corner in corners], width=3)

        # Draw the obstacles
        for obs in self.obstacles:
            (x_min, y_min), (x_max, y_max) = obs
            bottom_left = self.transform(np.array([x_min, y_min]))
            bottom_right = self.transform(np.array([x_max, y_min]))
            top_left = self.transform(np.array([x_min, y_max]))
            top_right = self.transform(np.array([x_max, y_max]))
            
            # Draw each obstacle as a polygon in the deformed space
            pygame.draw.polygon(self.screen, RED, [
                to_screen_coords(bottom_left),
                to_screen_coords(bottom_right),
                to_screen_coords(top_right),
                to_screen_coords(top_left)
            ])

        # Draw the agent
        agent_position = self.transform(self.state)
        pygame.draw.circle(self
        .screen, BLUE, to_screen_coords(agent_position), 10)

        # Draw the goal
        goal_position = self.transform(self.goal)
        pygame.draw.circle(self.screen, GREEN, to_screen_coords(goal_position), 12)

        # Draw observation radius as a dashed circle around the agent
        observation_radius = self.observation_radius * max(self.transformation_matrix.diagonal())
        pygame.draw.circle(self.screen, YELLOW, to_screen_coords(agent_position), 
                        int(observation_radius * scale), 1)

        # Update the display
        pygame.display.flip()

        # Handle key events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return
            elif event.type == pygame.KEYDOWN:
                # Press 'r' to reset environment
                if event.key == pygame.K_r:
                    self.reset()
                # Press 'w' to quit
                elif event.key == pygame.K_w:
                    pygame.quit()
                    return
                # Press 's' to save current state
                elif event.key == pygame.K_s:
                    self.save_state()
                # Press space to pause/resume
                elif event.key == pygame.K_SPACE:
                    self.pause()
                # Press arrow keys for manual control
                elif event.key == pygame.K_LEFT:
                    return self.step(3)  # Left action
                elif event.key == pygame.K_RIGHT:
                    return self.step(2)  # Right action
                elif event.key == pygame.K_UP:
                    return self.step(0)  # Up action
                elif event.key == pygame.K_DOWN:
                    return self.step(1)  # Down action

    def close(self):
        self.render_mode = None
        pygame.quit()
    
    def sample(self,num,limit):
        low,high = limit
        return low + np.random.rand(num)*(high-low)

In [ ]:
obstacles = [((0.14625, 0.3325), (0.565, 0.55625)), ((0.52875, 0.5375), (0.7375, 0.84125)), ((0.0, 0.00125), (0.01625, 0.99125)), ((0.0075, 0.00125), (0.99875, 0.04)), ((0.98875, 0.0075), (0.99875, 1.0)), ((0.00125, 0.9825), (0.99875, 1.0))]

env = ObservableDeformedGridworld(
    obstacles=obstacles, 
    render_mode="human"
)

state, observation = env.reset(np.random.randint(100))

done = False
while not done:
    try:
        env.render()
    except:
        break



In [ ]:
env.observe_obstacle()


In [ ]:
from stable_baselines3 import DQN
import numpy as np

obstacles = [((0.14625, 0.3325), (0.565, 0.55625)), 
             ((0.52875, 0.5375), (0.7375, 0.84125)), 
             ((0.0, 0.00125), (0.01625, 0.99125)), 
             ((0.0075, 0.00125), (0.99875, 0.04)), 
             ((0.98875, 0.0075), (0.99875, 1.0)), 
             ((0.00125, 0.9825), (0.99875, 1.0))]


def train_dqn(args):
    from stable_baselines3.common.callbacks import CheckpointCallback
    from wandb.integration.sb3 import WandbCallback
    import wandb

    total_timesteps = args.total_timesteps
    batch_size = args.batch_size
    lr = args.learning_rate
    target_update = args.target_update
    gamma = args.gamma

    config = {
        "policy_type": "MultiInputPolicy",
        "env_name": "ObservableDeformedGridworld",
        "total_timesteps": total_timesteps,
        "Batch_Size": batch_size,
        'grid_size': (1.0,1.0),
        'step_size': 0.1,
        'obstacles':obstacles,
        'observation_radius':0.2,
    }
    run = wandb.init(
        project="DQNsb3 - MDP - ObservableDeformedGridworld",
        config=config,
        sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
        monitor_gym=True,  # auto-upload the videos of agents playing the game
        save_code=True,  # optional
    )


    # Save a checkpoint every 10000 steps
    checkpoint_callback = CheckpointCallback(
                            save_freq=10000,
                            save_path=f"DQNsb3_{run.id}",
                            name_prefix="rl_model",
                            save_replay_buffer=False,
                            save_vecnormalize=True,
                        )

    callbacks = [ WandbCallback(
                                verbose=2,
                                log="parameters",
                                ),
                checkpoint_callback,
                ]


    from stable_baselines3.common.monitor import Monitor
    from stable_baselines3.common.vec_env import DummyVecEnv, VecVideoRecorder

    def make_env():
        env = ObservableDeformedGridworld(
            grid_size=(1.0, 1.0),
            step_size=0.1,
            goal=(0.9, 0.9),
            obstacles=obstacles,
            render_mode='human'
        )

        env = Monitor(env)  # record stats such as returns
        return env

    env = DummyVecEnv([make_env])


    model = DQN("MultiInputPolicy",env,batch_size=batch_size,gamma=gamma, 
                target_update_interval=target_update,verbose=2,
                tensorboard_log=f"runs/{run.id}", device="cpu", learning_rate=lr,
                train_freq=(1,"episode"), gradient_steps=1)
    model.learn(total_timesteps,progress_bar=True, callback=callbacks, log_interval=1)
    model.save(f"agents/pretrained/MDP/DQNsb3_{run.id}")
    env.close()
    run.finish()

if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser()
    
    parser.add_argument("--learning_rate", type=float, default=0.001)
    parser.add_argument("--batch_size", type=int, default=64)
    parser.add_argument("--total_timesteps", type=int, default=1000000) # env steps
    parser.add_argument("--target_update", type=int, default=555) # in env steps
    parser.add_argument("--gamma", type=float, default=0.99)
    parser.add_argument("--f", type=str, default=None)

    args = parser.parse_args()

    train_dqn(args)

Using cpu device
Logging to runs/zgjtuifm/DQN_1


Output()

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4        |
|    ep_rew_mean      | -2.02    |
|    exploration_rate | 1        |
| time/               |          |
|    episodes         | 1        |
|    fps              | 14       |
|    time_elapsed     | 0        |
|    total_timesteps  | 4        |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3        |
|    ep_rew_mean      | -2.01    |
|    exploration_rate | 1        |
| time/               |          |
|    episodes         | 2        |
|    fps              | 21       |
|    time_elapsed     | 0        |
|    total_timesteps  | 6        |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.33     |
|    ep_rew_mean      | -1.01    |
|    exploration_rate | 1        |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 7        |
|    ep_rew_mean      | -3.2     |
|    exploration_rate | 1        |
| time/               |          |
|    episodes         | 6        |
|    fps              | 116      |
|    time_elapsed     | 0        |
|    total_timesteps  | 42       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 7.86     |
|    ep_rew_mean      | -3.61    |
|    exploration_rate | 0.999    |
| time/               |          |
|    episodes         | 7        |
|    fps              | 141      |
|    time_elapsed     | 0        |
|    total_timesteps  | 55       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 7.12     |
|    ep_rew_mean      | -3.41    |
|    exploration_rate | 0.999    |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 8.18     |
|    ep_rew_mean      | -4.04    |
|    exploration_rate | 0.999    |
| time/               |          |
|    episodes         | 11       |
|    fps              | 189      |
|    time_elapsed     | 0        |
|    total_timesteps  | 90       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 8.17     |
|    ep_rew_mean      | -3.96    |
|    exploration_rate | 0.999    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 195      |
|    time_elapsed     | 0        |
|    total_timesteps  | 98       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 7.85     |
|    ep_rew_mean      | -3.81    |
|    exploration_rate | 0.999    |
| time/               |          |
|    episodes       

...

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 14.2     |
|    ep_rew_mean      | -7.53    |
|    exploration_rate | 0.626    |
| time/               |          |
|    episodes         | 3217     |
|    fps              | 344      |
|    time_elapsed     | 114      |
|    total_timesteps  | 39349    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.5      |
|    n_updates        | 3204     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 14.1     |
|    ep_rew_mean      | -7.46    |
|    exploration_rate | 0.626    |
| time/               |          |
|    episodes         | 3218     |
|    fps              | 344      |
|    time_elapsed     | 114      |
|    total_timesteps  | 39351    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.499    |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 13.1     |
|    ep_rew_mean      | -6.86    |
|    exploration_rate | 0.626    |
| time/               |          |
|    episodes         | 3224     |
|    fps              | 344      |
|    time_elapsed     | 114      |
|    total_timesteps  | 39382    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.536    |
|    n_updates        | 3211     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 12.9     |
|    ep_rew_mean      | -6.79    |
|    exploration_rate | 0.626    |
| time/               |          |
|    episodes         | 3225     |
|    fps              | 344      |
|    time_elapsed     | 114      |
|    total_timesteps  | 39388    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.535    |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 12.7     |
|    ep_rew_mean      | -6.63    |
|    exploration_rate | 0.626    |
| time/               |          |
|    episodes         | 3228     |
|    fps              | 344      |
|    time_elapsed     | 114      |
|    total_timesteps  | 39402    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.531    |
|    n_updates        | 3215     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 12.4     |
|    ep_rew_mean      | -6.51    |
|    exploration_rate | 0.626    |
| time/               |          |
|    episodes         | 3229     |
|    fps              | 344      |
|    time_elapsed     | 114      |
|    total_timesteps  | 39405    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.53     |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 12.6     |
|    ep_rew_mean      | -6.67    |
|    exploration_rate | 0.625    |
| time/               |          |
|    episodes         | 3232     |
|    fps              | 344      |
|    time_elapsed     | 114      |
|    total_timesteps  | 39438    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.64     |
|    n_updates        | 3219     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 12.7     |
|    ep_rew_mean      | -6.73    |
|    exploration_rate | 0.625    |
| time/               |          |
|    episodes         | 3233     |
|    fps              | 344      |
|    time_elapsed     | 114      |
|    total_timesteps  | 39450    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.634    |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 12.6     |
|    ep_rew_mean      | -6.71    |
|    exploration_rate | 0.625    |
| time/               |          |
|    episodes         | 3234     |
|    fps              | 344      |
|    time_elapsed     | 114      |
|    total_timesteps  | 39461    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.644    |
|    n_updates        | 3221     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 12.8     |
|    ep_rew_mean      | -6.74    |
|    exploration_rate | 0.625    |
| time/               |          |
|    episodes         | 3235     |
|    fps              | 344      |
|    time_elapsed     | 114      |
|    total_timesteps  | 39486    |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.651    |
|    n_updates      